# Langchain

In [1]:
import os

from langchain.agents import AgentExecutor, create_react_agent
from langchain_ollama import OllamaLLM

from travel_agent.qdrant import client as qdrant_client
from travel_agent.retrieval.langchain.prompt import SYSTEM_PROMPT
from travel_agent.retrieval.langchain.tool import GetExistingAvailableRubricsTool, TravelReviewQueryTool

if not os.getenv("QDRANT_URL"):
    print("Set QDRANT_URL and QDRANT_API_KEY env")
    exit(1)

In [3]:
MODEL_NAME = "hf.co/IlyaGusev/saiga_nemo_12b_gguf:Q4_0"
embed_model_name = "intfloat/multilingual-e5-base"
collection_name = "moskva_intfloat_multilingual_e5_base"


model = OllamaLLM(
    model=MODEL_NAME,
    base_url=os.getenv("OLLAMA_URL", "http://127.0.0.1:11434"),
)

get_available_rubrics_tool = GetExistingAvailableRubricsTool()
review_search_tool = TravelReviewQueryTool(embed_model_name, qdrant_client, collection_name)
tools = [get_available_rubrics_tool, review_search_tool]

agent = create_react_agent(llm=model, tools=tools, prompt=SYSTEM_PROMPT)
agent = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    max_iterations=10,
    handle_parsing_errors=True,
)

2025-04-26 23:16:37.227 | INFO     | travel_agent.retrieval.langchain.tool:__init__:91 - Using device: mps


In [4]:
result = agent.invoke({"input": "Посоветуй хорошее кафе в центре Москвы"})



> Entering new AgentExecutor chain...
Thought: 
Для начала нужно определить какие рубрики подходят для поиска кафе и где лучше всего искать отзывы о заведениях в центре Москвы.

Action:
get_existing_travel_review_rubrics

Action Input:
rubrics=["кафе", "центр", "Мoscow"]
['Кафе', 'Ресторан', 'Гостиница', 'Магазин продуктов', 'Супермаркет', 'Бар, паб', 'Салон красоты', 'Быстрое питание', 'Доставка еды и обедов', 'Ногтевая студия', 'Автосервис, автотехцентр', 'Кофейня', 'Пиццерия', 'Парикмахерская', 'Музей', 'Банкетный зал', 'Парк культуры и отдыха', 'Медцентр, клиника', 'Торговый центр', 'Развлекательный центр', 'Косметология', 'Стоматологическая клиника', 'Суши-бар', 'Достопримечательность', 'Магазин хозтоваров и бытовой химии', 'Кондитерская', 'Магазин автозапчастей и автотоваров', 'Пекарня', 'Столовая', 'Фитнес-клуб', 'Шиномонтаж', 'Массажный салон', 'Кальян-бар', 'Пункт выдачи', 'Аптека', 'Салон бровей и ресниц', 'Бассейн', 'Магазин цветов', 'Магазин алкогольных напитков', 'База, 

In [5]:
print(result["output"])

["=== Отзыв на Il Letterato ===\nАдрес: Москва, улица Большая Дмитровка, 8с2\nРейтинг: 5.0\nКатегории: ['Ресторан', 'Кафе', 'Пиццерия']\nТекст: Уютное место, приятная атмосфера, вежливый персонал. Вкусное меню и недорого для центра Москвы. Вкусный кофе ☕️  \n\n", "=== Отзыв на Неаполь ===\nАдрес: Москва, Вознесенский переулок, 12, стр. 3\nРейтинг: 5.0\nКатегории: ['Гостиница']\nТекст: Хорошая уютная гостиница в центре Москвы \n\n", "=== Отзыв на Kiosk 1936 ===\nАдрес: Москва, Чистопрудный бульвар, 12, корп. 4\nРейтинг: 5.0\nКатегории: ['Кафе', 'Ресторан', 'Кофейня']\nТекст: Место с пленяющей атмосферой, в котором Москва ощущается иначе. Кроме того, что здесь вкусный кофе и чудные завтраки, приятно также то, что заведение классно отражает идею особенного киоска в одном из множества стран и городов, в котором хочется остановиться, позавтракать, почитать, побыть в моменте наедине с собой и чемто чарующим. Спасибо ребятам за труд, сюда хочется возвращаться, в какой бы части города или стра